In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
import h5py
from google.colab import files
uploaded = files.upload()

Saving test.hdf5 to test.hdf5
Saving train.hdf5 to train.hdf5


In [ ]:
f1 = h5py.File("train.hdf5","r")
X_train = np.array(f1["image"])/255
Y_train = np.array([f1["label"]])
X_train = np.reshape(X_train,(60000,784)).T

In [ ]:
f2 = h5py.File("test.hdf5","r")
X_test = np.array(f2["image"])/255
Y_test = np.array([f2["label"]])
shape = (Y_test.size, Y_test.max()+1)
one_hot = np.zeros(shape)
rows = np.arange(Y_test.size)
one_hot[rows, Y_test] = 1
Y_test = one_hot.T
X_test = np.reshape(X_test,(10000,784)).T

In [ ]:
def sigmoid(z) :
  return (1/(1+np.exp(-z)))
def relu(X) :
  X[X<0]=0
  return X
shape = (Y_train.size, Y_train.max()+1)
one_hot = np.zeros(shape)
rows = np.arange(Y_train.size)
one_hot[rows, Y_train] = 1
Y_train = one_hot.T
shuffle_index = np.random.permutation(X_train.shape[1])
X_train,Y_train = X_train[:,shuffle_index] , Y_train[:,shuffle_index]
print(X_train.shape)
print(Y_train.shape)
n = X_train.shape[0]
m = X_train.shape[1]

(784, 60000)
(10, 60000)


In [ ]:
def softmax(x) :
  x = x - np.max(x)
  return np.exp(x) / np.sum(np.exp(x), axis=0,keepdims=True)
def relu(X) :
  X[X<0]=0
  return X
def initialize_wts(struct) :
  W = {}
  b = {}
  for i in range(1,len(struct)) :
    W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*np.sqrt(2/struct[i-1])
    #W["W" + str(i)] = np.random.randn(struct[i],struct[i-1])*0.01
    #W["W" + str(i)] = np.random.rand(struct[i],struct[i-1])
    b["b" + str(i)] = np.zeros((struct[i],1))
  return W,b
def forward_prop(X_train,W,b) :
  return relu(np.dot(W,X_train) + b)

In [ ]:
def model(struct,X_train,Y_train,lr,W,b,vdw,vdb,sdw,sdb,laa,num_itr) :
  n = X_train.shape[0]
  m = X_train.shape[1]
  beta = 0.8
  beta2 = 0.8
  epsilon = 1e-8
  cost1 = 50
  for k in range(0,num_itr) :
    A = {}
    A["A" + str(0)] = np.copy(X_train)
    for i in range(1,len(struct)-1) :
      A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
    A["A" + str(len(struct) - 1)] = softmax(np.dot(W["W" + str(len(struct) - 1)] , A["A" + str(len(struct) - 2)]) + b["b" + str(len(struct) - 1)])
    dW = {}
    db = {}
    dZ = {}
    dZ["dZ" + str(len(struct) - 1)] = A["A" + str(len(struct) - 1)] - Y_train
    dW["dW" + str(len(struct) - 1)] = (1/m)*(np.dot(dZ["dZ" + str(len(struct) - 1)],A["A" + str(len(struct) - 2)].T))
    db["db" + str(len(struct) - 1)] = (1/m)*(np.sum(dZ["dZ" + str(len(struct) - 1)],axis = 1, keepdims = True))
    for i in range(0,len(W) - 1) :
      dZ["dZ" + str(len(W) - 1 - i)] = np.dot(W["W" + str(len(W) - i)].T,dZ["dZ" + str(len(W) - i)])*np.int64(A["A" + str(len(W) - 1 - i)] > 0)
      dW["dW" + str(len(W) - 1 - i)] = (1/m)*np.dot(dZ["dZ" + str(len(W) - 1 - i)] , A["A" + str(len(W) - 1 - i - 1)].T)
      db["db" + str(len(W) - i - 1)] = (1/m)*np.sum(dZ["dZ" + str(len(W) - 1 - i)],axis = 1,keepdims = True)
    for i in range(1,len(W) + 1) :
      vdw["vdw" + str(i)] = (beta*vdw["vdw" + str(i)] + (1-beta)*dW["dW" + str(i)])/(1 - beta**(laa+1))
      vdb["vdb" + str(i)] = (beta*vdb["vdb" + str(i)] + (1-beta)*db["db" + str(i)])/(1 - beta**(laa+1))
      sdw["sdw" + str(i)] = (beta2*sdw["sdw" + str(i)] + (1-beta2)*dW["dW" + str(i)]*dW["dW" + str(i)])/(1 - beta2**(laa+1))
      sdb["sdb" + str(i)] = (beta2*sdb["sdb" + str(i)] + (1-beta2)*db["db" + str(i)]*db["db" + str(i)])/(1 - beta2**(laa+1))
      #vdw["vdw" + str(i)] = beta*vdw["vdw" + str(i)] + (1-beta)*dW["dW" + str(i)]
      #vdb["vdb" + str(i)] = beta*vdb["vdb" + str(i)] + (1-beta)*db["db" + str(i)]
      #sdw["sdw" + str(i)] = beta2*sdw["sdw" + str(i)] + (1-beta2)*dW["dW" + str(i)]*dW["dW" + str(i)]
      #sdb["sdb" + str(i)] = beta2*sdb["sdb" + str(i)] + (1-beta2)*db["db" + str(i)]*db["db" + str(i)]
      W["W" + str(i)] = W["W" + str(i)] - lr*(vdw["vdw" + str(i)]/(np.sqrt(sdw["sdw" + str(i)]) + epsilon))
      b["b" + str(i)] = b["b" + str(i)] - lr*(vdb["vdb" + str(i)]/(np.sqrt(sdb["sdb" + str(i)]) + epsilon))
    cost = (-1/m)*(np.dot(Y_train,np.log(A["A" + str(len(struct) - 1)]).T) + np.dot(1 - Y_train,np.log(1 - A["A" + str(len(struct) - 1)]).T))
    #cost1 = (-1/m)*np.sum(np.dot(Y_train,np.log(A["A" + str(len(struct) - 1)]).T) + np.dot(1 - Y_train,np.log(1 - A["A" + str(len(struct) - 1)]).T))
    cost1 = -np.mean(Y_train * np.log(A["A" + str(len(struct) - 1)] + 1e-8))
    #log_likelihood = -np.log(A["A" + str(len(struct) - 1)][range(m),Y_train])
    #cost1 = np.sum(log_likelihood) / m
    #costu.append(float(cost1))
    #print("Cost after {}th iteration is : {} {}".format(k,cost1,lr))
  #plt.plot(list(range(num_itr)) , costu)
  #plt.show()
  return W,b,A["A" + str(len(struct) - 1)],float(cost1),vdw,vdb,sdw,sdb

In [ ]:
def train(struct,X_train,Y_train,lr,epoks,batch_size):
  costu = []
  W,b = initialize_wts(struct)
  vdw = {}
  vdb = {}
  sdw = {}
  sdb = {}
  for i in range(1,len(W) + 1) :
    vdw["vdw" + str(i)] = 0
    vdb["vdb" + str(i)] = 0
    sdw["sdw" + str(i)] = 0
    sdb["sdb" + str(i)] = 0
  l = 0
  for j in range(0,epoks) :
    for i in range(0,int(X_train.shape[1]/batch_size)+1) :
      if i != int(X_train.shape[1]/batch_size) :
        X_batch = X_train[:,i*batch_size:(i+1)*batch_size]
        Y_batch = Y_train[:,i*batch_size:(i+1)*batch_size]
        lim = (i+1)*batch_size
      elif i*batch_size == X_train.shape[1] :
        continue
      else :
        X_batch = X_train[:,i*batch_size:-1]
        Y_batch = Y_train[:,i*batch_size:-1]
        lim = X_train.shape[1]
      W,b,A,cost1,vdw,vdb,sdw,sdb = model(struct , X_batch , Y_batch , lr ,W,b,vdw,vdb,sdw,sdb,l,num_itr = 1)
      costu.append(cost1)
      print("Epok : {}  Data_Passed : {}/{}  Train_Loss : {}".format(j+1,lim,X_train.shape[1],cost1))
      l = l + 1
  A = {}
  A["A" + str(0)] = np.copy(X_train)
  for i in range(1,len(struct)-1) :
    A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
  A = softmax(np.dot(W["W" + str(len(struct) - 1)] , A["A" + str(len(struct) - 2)]) + b["b" + str(len(struct) - 1)])
  plt.plot(list(range(len(costu))),costu)
  return W,b,A

In [ ]:
W,b,A = train([n,600,400,200,100,50,10],X_train,Y_train,0.001,5,64)

In [ ]:
def get_output(X,W,b) :
  struct = len(W) + 1
  A = {}
  A["A" + str(0)] = np.copy(X)
  for i in range(1,struct-1) :
    A["A" + str(i)] = forward_prop(A["A" + str(i-1)],W["W" + str(i)],b["b" + str(i)])
  A["A" + str(struct - 1)] = softmax(np.dot(W["W" + str(struct - 1)] , A["A" + str(struct - 2)]) + b["b" + str(struct - 1)])
  return A["A" + str(struct - 1)]

In [ ]:
def predict(A,Y) :
  for i in range(0,A.shape[1]) :
    a = np.max(A[:,i])
    A[:,i] = (A[:,i] == a)
  fai = np.absolute(A - Y)
  count = 0
  for i in range(0,fai.shape[1]) :
    if np.sum(fai[:,i]) == 0 :
      count += 1
  return (count/A.shape[1])*100

In [ ]:
A_train = get_output(X_train,W,b)
A_test  = get_output(X_test,W,b)
print("Train accuracy is {}".format(predict(A_train,Y_train)))
print("Test accuracy is {}".format(predict(A_test,Y_test)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Train accuracy is 98.19333333333333
Test accuracy is 96.95
